In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# ---------------------------------------------------------
# 1. 데이터 로드 및 기초 병합
# ---------------------------------------------------------

path_approval = './카드승인정보_WideFormat_50000_2569.csv'
path_balance = './카드잔액정보_WideFormat_50000_487.csv'

months_6m = ['07', '08', '09', '10', '11', '12']
months_3m = ['10', '11', '12']

# 승인정보 로드
cols_app = ['발급회원번호'] + [f'이용금액_신용_B0M_{m}' for m in months_6m] + [f'이용건수_신용_B0M_{m}' for m in months_6m]
df_app = pd.read_csv(path_approval, usecols=lambda x: x in cols_app).fillna(0)

# 잔액정보 로드
df_bal_raw = pd.read_csv(path_balance)
cols_bal = ['발급회원번호'] + [f'잔액_B0M_{m}' for m in months_6m] + [f'잔액_현금서비스_B0M_{m}' for m in months_6m] + \
           [f'잔액_카드론_B0M_{m}' for m in months_6m] + [f'연체잔액_B0M_{m}' for m in months_6m] + [f'월중평잔_{m}' for m in months_6m]
exist_bal = [c for c in cols_bal if c in df_bal_raw.columns]
df_bal = df_bal_raw[exist_bal].fillna(0)

df_score = pd.merge(df_app, df_bal, on='발급회원번호', how='inner')

# ---------------------------------------------------------
# 2. 점수 및 타겟(Target) 생성 로직
# ---------------------------------------------------------

# 상태 점수 계산
df_score['Score_BadDebt'] = (((df_score['잔액_현금서비스_B0M_12'] - df_score['잔액_현금서비스_B0M_11']) / (df_score['잔액_현금서비스_B0M_11'] + 1) * 1.5) +
                            ((df_score['잔액_카드론_B0M_12'] - df_score['잔액_카드론_B0M_11']) / (df_score['잔액_카드론_B0M_11'] + 1) * 1.0))
df_score['Score_Delinq'] = (df_score['연체잔액_B0M_12'] * 3.0) + (df_score['연체잔액_B0M_11'] * 2.0) + (df_score['연체잔액_B0M_10'] * 1.0)
df_score['Score_Activity'] = ((df_score[[f'이용건수_신용_B0M_{m}' for m in months_3m]].sum(axis=1) * 2) - 
                               df_score[[f'이용건수_신용_B0M_{m}' for m in months_6m]].sum(axis=1)) / \
                              (df_score[[f'이용건수_신용_B0M_{m}' for m in months_6m]].sum(axis=1) + 1) * 100
df_score['Score_Asset'] = (df_score[[f'월중평잔_{m}' for m in months_3m if f'월중평잔_{m}' in df_score.columns]].mean(axis=1) / \
                           (df_score[[f'월중평잔_{m}' for m in months_6m if f'월중평잔_{m}' in df_score.columns]].mean(axis=1) + 1)) * 10
df_score['Score_Status_Total'] = (df_score['Score_BadDebt'] + df_score['Score_Delinq']) - (df_score['Score_Activity'] + df_score['Score_Asset'])

# 기울기 점수 계산
def get_slope(row, prefix):
    cols = [f"{prefix}_{m}" for m in months_6m]
    y = row[cols].values.astype(float)
    if np.sum(y) == 0: return 0
    return linregress(np.arange(len(y)), y)[0]

df_score['Slope_Spend'] = df_score.apply(lambda r: get_slope(r, '이용금액_신용_B0M'), axis=1)
df_score['Slope_Balance'] = df_score.apply(lambda r: get_slope(r, '잔액_B0M'), axis=1)
df_score['Slope_Count'] = df_score.apply(lambda r: get_slope(r, '이용건수_신용_B0M'), axis=1)

scaler = MinMaxScaler()
def norm_risk(s):
    risk = s.apply(lambda x: -x if x < 0 else 0)
    return scaler.fit_transform(risk.values.reshape(-1, 1)).flatten()

df_score['Score_Slope_Total'] = (norm_risk(df_score['Slope_Spend']) * 40 + norm_risk(df_score['Slope_Balance']) * 30 + norm_risk(df_score['Slope_Count']) * 30)
df_score['Final_Total_Score'] = (df_score['Score_Status_Total'].fillna(0) + df_score['Score_Slope_Total'].fillna(0)) * 0.5

# Risk_Count 기반 타겟 정의
df_le0 = df_score[(df_score['Slope_Spend'] <= 0) & (df_score['Slope_Balance'] <= 0) & (df_score['Slope_Count'] <= 0)].copy()
df_le0['Risk_Count'] = (df_le0[['Score_BadDebt', 'Score_Delinq']].gt(0).sum(axis=1) + 
                        df_le0['Score_Activity'].lt(0).astype(int) + 
                        df_le0['Score_Asset'].eq(0).astype(int))

churn_list = df_le0[df_le0['Risk_Count'] >= 2]['발급회원번호'].unique()
df_score['Target'] = 0
df_score.loc[df_score['발급회원번호'].isin(churn_list), 'Target'] = 1

# ---------------------------------------------------------
# 3. 원본 전체 데이터 병합 및 최종 확인
# ---------------------------------------------------------

path_raw = './50k_wideformat_all_50000_4921.csv'
df_raw = pd.read_csv(path_raw, low_memory=False)

# 병합 시 Target 컬럼 중복으로 인한 Target_x 발생을 방지하기 위해 
# 원본 데이터에 Target 컬럼이 있다면 미리 제거함
if 'Target' in df_raw.columns:
    df_raw = df_raw.drop(columns=['Target'])

# 스코어 데이터와 원본 데이터 병합
df_final = pd.merge(df_score[['발급회원번호', 'Final_Total_Score', 'Score_Status_Total', 'Score_Slope_Total', 'Target']], 
                     df_raw, 
                     on='발급회원번호', 
                     how='inner')

# 결과 확인 (이제 'Target' 컬럼을 안전하게 호출할 수 있음)
print("✅ 최종 데이터셋 구성 완료")
df_final.info()
print("-" * 30)
print(df_final['Target'].value_counts(normalize=True))

✅ 최종 데이터셋 구성 완료
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 4925 entries, 발급회원번호 to 혜택수혜율_B0M_12
dtypes: float64(1467), int64(3271), object(187)
memory usage: 1.8+ GB
------------------------------
Target
0    0.94738
1    0.05262
Name: proportion, dtype: float64


In [2]:
# ---------------------------------------------------------
# 4. 최종 데이터셋 CSV 파일 저장
# ---------------------------------------------------------

# 한글 깨짐 방지를 위해 utf-8-sig 인코딩을 사용하며, 인덱스는 제외하고 저장함
df_final.to_csv('Target_data_산출.csv', index=False, encoding='utf-8-sig')

In [3]:
df_final['Target'].value_counts(normalize=True)

Target
0    0.94738
1    0.05262
Name: proportion, dtype: float64